In [ ]:
import numpy
import scipy
import scipy.misc
import scipy.io
import algopy
from algopy import UTPM

import time

### Utility functions

In [ ]:
# i!/(i-k)!
def falling_factorial(k, i):
    return scipy.special.poch(i - k + 1, k)

def UTP_deriv(x, k):
    n = len(x)
    fact = falling_factorial(k, numpy.arange(n))
    x = x * fact

    # shift by k places
    y = numpy.zeros(n-k)
    y[:n-k] = x[k:]
    
    return y
    
x = numpy.array([1, 2, 3, 4])

d  = UTP_deriv(x, 1)
d2 = UTP_deriv(d, 1)

d2_prime = UTP_deriv(x, 2)

print d
print d2
print d2_prime

UTP PGFFA
=========

### Defining data/loading data

In [ ]:
y = numpy.array([3,4,5])
K = len(y)
Y = sum(y)
Lambda = numpy.array([8, 10, 12])
Delta = numpy.array([0.6, 0.4])
Rho = numpy.array([0.8, 0.8, 0.8])

### Indeterminate Determination

In [ ]:
def UTP_Reverse(s_K, K, Rho, Delta):
    S = numpy.zeros(K); S[K-1] = s_K
    U = numpy.zeros(K)
    V = numpy.zeros(K)
    for k in range(K-1,-1,-1):
        V[k] = S[k] * (1 - Rho[k])
        U[k] = V[k]
        if k > 0:
            S[k-1] = Delta[k-1] * U[k] + 1 - Delta[k-1]
            
    return S, U, V
        
#test reverse mode
S, U, V = UTP_Reverse(1, K, Rho, Delta)

print "S: ", S
print "U: ", U
print "V: ", V

### UTP PGFFA

In [ ]:
def Gamma_UTP(Psi_k, lambda_k, V_k):
    D = len(Psi_k.data)
    
    #setup the indeterminate variable
    v_k = UTPM(numpy.zeros((D,1)))
    v_k.data[0,0] = V_k
    
    if D > 1:
        v_k.data[1,0] = 1
    
    Gamma_k = Psi_k * numpy.exp(lambda_k * (v_k - 1))
    
    return Gamma_k

In [ ]:
def Alpha_UTP(Gamma_k, rho_k, y_k, S_k):
    D = len(Gamma_k.data)
    D_prime = D - y_k
    
    #setup the indeterminate variable
    s_k = UTPM(numpy.zeros((D_prime,1)))
    s_k.data[0,0] = S_k
    
    if D_prime > 1:
        s_k.data[1,0] = 1
    
    #initialize a dummy UTP for Alpha
    Alpha_k = UTPM(numpy.zeros((D_prime,1)))
    
    Alpha_k.data[:,0] = UTP_deriv(Gamma_k.data[:,0], y_k)
    
    Alpha_k.data[:,0] = Alpha_k.data[:,0] * numpy.power(1 - rho_k, numpy.arange(D_prime))
    
    Alpha_k = Alpha_k * 1./scipy.misc.factorial(y_k) * numpy.power(s_k * rho_k, y_k)
    
    return Alpha_k

In [ ]:
#j subscript here is equivalent to k-1
def Psi_UTP(Alpha_j, delta_j, U_k):
    D = len(Alpha_j.data)
    
    #setup the indeterminate variable
    u_k = UTPM(numpy.zeros((D,1)))
    u_k.data[0,0] = U_k
    
    if D > 1:
        u_k.data[1,0] = 1

    #chain rule
    Psi_k = Alpha_j.copy()
    Psi_k.data[:,0] = Psi_k.data[:,0] * numpy.power(delta_j, numpy.arange(D))
    
    return Psi_k

In [ ]:
def UTP_PGFFA(y, Lambda, Delta, Rho, d=1):
    assert d >= 1
    
    K = len(y)
    Y = sum(y)
    
    S, U, V = UTP_Reverse(1, K, Rho, Delta)
    
    #allocate/wipe message storage
    Gamma = [None] * K
    Alpha = [None] * K
    Psi   = [None] * K
    
    #initialize first Psi (survivor) message to 1
    Psi[0] = UTPM(numpy.zeros((Y + d,1)))
    Psi[0].data[0,0] = 1
    
    for k in range(0,K):
        Gamma[k] = Gamma_UTP(Psi[k], Lambda[k], V[k])
        
        Alpha[k] = Alpha_UTP(Gamma[k], Rho[k], y[k], S[k])
        
        if k < K - 1:
            Psi[k+1] = Psi_UTP(Alpha[k], Delta[k], U[k])
    
    return Alpha, Gamma, Psi

Alpha, Gamma, Psi = UTP_PGFFA(y, Lambda, Delta, Rho)
print Alpha[-1].data

Alpha, Gamma, Psi = UTP_PGFFA(y, Lambda, Delta, Rho, d=3)
print Alpha[-1].data

In [ ]:
def UTPPGF_mean(F):
    return F.data[1]
def UTPPGF_var(F):
    return (2 * F.data[2]) - numpy.power(F.data[1], 2) + F.data[1]

### Correctness test vs Matlab

In [ ]:
y = numpy.array([3,4,5])
K = len(y)
Y = sum(y)
Lambda = numpy.array([8, 10, 12])
Delta = numpy.array([0.6, 0.4])
Rho = numpy.array([0.8, 0.8, 0.8])

Alpha, Gamma, Psi = UTP_PGFFA(y, Lambda, Delta, Rho, d=3)

print "ll:\t%e"   % Alpha[-1].data[0]
print "Mean:\t%e" % UTPPGF_mean(Alpha[-1])
print "Var:\t%e"  % UTPPGF_var(Alpha[-1])

print "\n", "Compare with the following Matlab code:"
print "y =", y, ";"
print "lambda =", Lambda, ";"
print "delta =", Delta, ";"
print "rho =", Rho[0], ";"
print "[ll, ~, ~, ~, messages] = gf_forward(y, lambda, rho, delta);"
print "[mean, var] = moments_pgf(messages(end).f, messages(end).a, messages(end).b);"
print "fprintf('ll:\\t %.06e\\n', exp(ll));"
print "fprintf('mean:\\t %.06e\\n', mean);"
print "fprintf('mean:\\t %.06e\\n', var);"

### Runtime test vs Matlab

In [ ]:
import time

y = numpy.array([6,8,10,6,8,10,6,8,10])
K = len(y)
Y = sum(y)
Lambda = numpy.array([16, 20, 24, 16, 20, 24, 16, 20, 24])
Delta = numpy.array([0.6, 0.4, 0.6, 0.4, 0.6, 0.4, 0.6, 0.4])
Rho = numpy.array([0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8])

reps = 100
t_start = time.clock()
for i in range(0,reps):
    Alpha, Gamma, Psi = UTP_PGFFA(y, Lambda, Delta, Rho, d=1)
total_time = time.clock() - t_start

print total_time / reps

print "\n", "Compare with the following Matlab code:"
print "y =", y, ";"
print "lambda =", Lambda, ";"
print "delta =", Delta, ";"
print "rho =", Rho[0], ";"
print "nIter =", reps, ";"
print "tStart = tic;"
print "for i = 1:nIter"
print "ll = gf_forward(y, lambda, rho, delta);"
print "end"
print "tTotal = toc(tStart)/nIter;"
print "fprintf('elapsed time: %0.8f\\n', tTotal)"

### Runtime test w/ Matlab data

In [ ]:
def utppgffa_vs_matlab(resultdir):
    #read in the data saved out from a matlab experiment
    data = scipy.io.loadmat(os.path.join(resultdir, "data.mat"))
    y      = data.get('y')
    Lambda = data.get('lambda')
    Rho    = numpy.squeeze(data.get('rho'))
    Delta  = numpy.squeeze(data.get('delta'))
    
    print y.shape
    print Lambda.shape
    print Rho.shape
    print Delta.shape
    
    #dimensions
    #nN and nRho were the grid search dimensions
    #nIter is how many times each experiment was repeated
    #K is the number of observations in each sample
    [nN, nRho, nIter, K] = y.shape
    
    #repeat each experiment using UTPPGFFA
    runtime = numpy.zeros((nN, nRho, nIter))
    for iN in range(0, nN): #indexing through rows of lambda
        for iRho in range(0, nRho):
            for iIter in range(0, nIter):
                print "Experiment %d of %d, iter %d of %d" % ((iRho + 1) + iN * nRho,
                                                              nN * nRho,
                                                              iIter,
                                                              nIter)
                t_start = time.clock()
                [Alpha, Gamma, Psi] = UTP_PGFFA(y[iN, iRho, iIter, ],
                                                Lambda[iN, ],
                                                Delta,
                                                Rho[iRho] * numpy.ones(K),
                                                d = 1)
                runtime[iN, iRho, iIter] = time.clock() - t_start
    return runtime
    
resultdir = "/Users/kwinner/Work/Data/Results/20170111T134734854"
runtime = utppgffa_vs_matlab(resultdir)
scipy.io.savemat(os.path.join(resultdir, "utppgffaruntime.mat"),
                 {'runtime':runtime})

### Test space

In [ ]:
print numpy.zeros((5,4))

UTPM example
------------

In [ ]:
from numpy import sin,cos,exp

def f(x):
    return exp(3*x)

D = 10; P = 1
x = UTPM(numpy.zeros((D,P)))
x.data[0,0] = 1
x.data[1,0] = 1

y = f(x)
print("coefficients of y=", y.data[:,0])

print ", ".join("%s: %s" % item for item in vars(x).items())
# print ", ".join("%s: %s" % item for item in dir(x).items())
dir(x)

2 state UTPPGFFA
------------

In [ ]:
# two state chain
utpm_coeff_template = numpy.zeros((Y,1))
utpm_coeff_template[0,0] = 1
utpm_coeff_template[1,0] = 1

### GAMMA_1 MESSAGE ###
v_1 = UTPM(numpy.copy(utpm_coeff_template))
v_1.data[0,0] = V[0]
# Gamma_1 = v_1 - 1
# Gamma_1 = Lambda[0] * (v_1 - 1)
Gamma_1 = numpy.exp(Lambda[0] * (v_1 - 1))

### ALPHA_1 MESSAGE ###
s_1 = UTPM(numpy.copy(utpm_coeff_template))
s_1.data[0,0] = S[0]

Alpha_1 = UTPM(numpy.copy(utpm_coeff_template))
Alpha_1.data[:,0] = deriv(Gamma_1.data[:,0], y[0])

#manual implementation of chain rule g(s_1) = s_1(1-rho_1)
#evaluation of Gamma(s_1(1-rho_1)) already "happened" in backward pass
Alpha_1.data[:,0] = Alpha_1.data[:,0] * numpy.power(1 - Rho[0], numpy.arange(Y))

#multiply by 1/y_1!(s_1 rho_1)^{y_1}
Alpha_1 = Alpha_1 * 1./scipy.misc.factorial(y[0]) * numpy.power(s_1 * Rho[0], y[0])

### PSI_2 MESSAGE ###
u_2 = UTPM(numpy.copy(utpm_coeff_template))
u_2.data[0,0] = U[1]

#manual implementation of chain rule g(u_2) = delta_1 * u_2 + 1 - delta_1
Psi_2 = Alpha_1.copy()
Psi_2.data[:,0] = Psi_2.data[:,0] * numpy.power(Delta[0], numpy.arange(Y))

### GAMMA_2 MESSAGE ###
v_2 = UTPM(numpy.copy(utpm_coeff_template))
v_2.data[0,0] = V[1]
Gamma_2 = Psi_2 * numpy.exp(Lambda[1] * (v_2 - 1))

### ALPHA_2 MESSAGE ###
s_2 = UTPM(numpy.copy(utpm_coeff_template))
s_2.data[0,0] = S[1]
Alpha_2 = UTPM(numpy.copy(utpm_coeff_template))
Alpha_2.data[:,0] = deriv(Gamma_2.data[:,0], y[1])

#### DAN added these two lines !
Alpha_2.data[:,0] = Alpha_2.data[:,0] * numpy.power(1 - Rho[1], numpy.arange(Y))
Alpha_2 = Alpha_2 * 1./scipy.misc.factorial(y[1]) * numpy.power(s_2 * Rho[1], y[1])

#test forward
print "v_1: ", V[0]
print "Gamma_1: ", Gamma_1
print "s_1: ", S[0]
print "Alpha_1: ", Alpha_1
print "u_2: ", U[1]
print "Psi_2: ", Psi_2
print "v_2: ", V[1]
print "Gamma_2: ", Gamma_2
print "s_2: ", S[1]
print "Alpha_2: ", Alpha_2